# Prompt, Mask, Train, Save, Pipeline-Predict

#### prompt template: 2

# Notebook:

##### Imports
##### replace 'MajorClaim' by 'Stance'
##### make train and test datasets
##### train BERT Mask on the train dataset
##### save the model
##### create a pipeline using the saved model
##### run predictions on it
##### evaluate with the test set

### Imports

In [1]:
!pip install transformers
!pip install ipywidgets
!pip install IProgress
!pip install pandas
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
import transformers
import torch

In [3]:
from transformers import BertTokenizer, BertForMaskedLM

import numpy as np

import datasets
from datasets import Dataset
from datasets import ClassLabel
from datasets import load_metric

In [4]:
import pandas as pd

### tokenizer

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

### make the train, test dataframe

In [6]:
df = pd.read_pickle("/notebooks/Prompting/notebooks/pe_dataset_w_prompts_1_2_3_pickle")

In [7]:
# rename MajorClaim as stance

In [8]:
df

,essay_nr,component_id,label_and_comp_idxs,text,label_x,label_ComponentType,relation_SupportAttack,label_RelationType,label_LinkedNotLinked,split,...,nr_following_comps_in_para,structural_fts_as_text,structural_fts_as_text_combined,para_ratio,first_or_last,strct_fts_w_position_in_essay,component_pos_tags,strct_fts_essay_position_pos_tags,prompted_representation_2,prompted_representation_3
0,essay001,T1,MajorClaim 503 575,we should attach more importance to cooperatio...,MajorClaim,MajorClaim,[],,Linked,TRAIN,...,0,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.25,1,Topic: Should students be taught to compete or...,"Part Of Speech tags: PRON, VERB, VERB, ADJ, NO...",Topic: Should students be taught to compete or...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl..."
1,essay001,T2,MajorClaim 2154 2231,a more cooperative attitudes towards life is m...,MajorClaim,MajorClaim,[],,Linked,TRAIN,...,0,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,1.00,1,Topic: Should students be taught to compete or...,"Part Of Speech tags: DET, ADV, ADJ, NOUN, ADP,...",Topic: Should students be taught to compete or...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl..."
2,essay001,T3,Claim 591 714,"through cooperation, children can learn about ...",Claim,Claim,[],Support,Linked,TRAIN,...,3,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.50,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: ADP, NOUN, PUNCT, NOUN, V...",Topic: Should students be taught to compete or...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl..."
3,essay001,T4,Premise 716 851,What we acquired from team work is not only ho...,Premise,Premise,[],Support,NotLinked,TRAIN,...,2,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.50,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: PRON, PRON, VERB, ADP, NO...",Topic: Should students be taught to compete or...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl..."
4,essay001,T5,Premise 853 1086,"During the process of cooperation, children ca...",Premise,Premise,[],Support,NotLinked,TRAIN,...,1,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.50,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: ADP, DET, NOUN, ADP, NOUN...",Topic: Should students be taught to compete or...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5968,essay402,T11,Premise 1275 1339,indirectly they will learn how to socialize ea...,Premise,Premise,[],Support,NotLinked,TRAIN,...,3,Topic: Children should studying hard or playin...,Topic: Children should studying hard or playin...,0.75,0,Topic: Children should studying hard or playin...,"Part Of Speech tags: ADV, PRON, VERB, VERB, AD...",Topic: Children should studying hard or playin...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl..."
5969,essay402,T12,Premise 1341 1388,That will make children getting lots of friends,Premise,Premise,[],Support,NotLinked,TRAIN,...,2,Topic: Children should studying hard or playin...,Topic: Children should studying hard or playin...,0.75,0,Topic: Children should studying hard or playin...,"Part Of Speech tags: DET, VERB, VERB, NOUN, VE...",Topic: Children should studying hard or playin...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl..."
5970,essay402,T13,Premise 1393 1436,they can contribute positively to communi

In [9]:
def rename_MC(x):
    
    prompted_rep = x.prompted_representation_3
    
    last_token = prompted_rep.split()[-1]
    
    if last_token == 'MajorClaim.':
    
        new_rep = prompted_rep.replace(last_token, "Stance.")
        
    else:
        
        new_rep = prompted_rep
    
    return new_rep
    
    

In [10]:
rename_MC(df.iloc[0])

'How is the component best described?: "MajorClaim", "Claim" or "Premise". Component: Topic: Should students be taught to compete or to cooperate?, Sentence: From this point of view, I firmly believe that we should attach more importance to cooperation during primary education., First or last in essay: Yes, First in paragraph: Yes, Last in paragraph: Yes, In in introduction: Yes, Is in conclusion: No. Part Of Speech tags: PRON, VERB, VERB, ADJ, NOUN, ADP, NOUN, ADP, ADJ, NOUN. Stance.'

In [11]:
# correct now. now run it on the whole df.

In [12]:
df['prompted_rep_3_w_modif_MC'] = df.apply(lambda x: rename_MC(x), axis=1)

In [14]:
df['prompted_rep_3_w_modif_MC'][0]

'How is the component best described?: "MajorClaim", "Claim" or "Premise". Component: Topic: Should students be taught to compete or to cooperate?, Sentence: From this point of view, I firmly believe that we should attach more importance to cooperation during primary education., First or last in essay: Yes, First in paragraph: Yes, Last in paragraph: Yes, In in introduction: Yes, Is in conclusion: No. Part Of Speech tags: PRON, VERB, VERB, ADJ, NOUN, ADP, NOUN, ADP, ADJ, NOUN. Stance.'

In [14]:
# correct the major claim? --> stance business

In [15]:
def replace_mc(x):
    
    sent = x.prompted_rep_3_w_modif_MC
    new_sent = sent.replace('How is the component best described?: "MajorClaim"', 'How is the component best described?: "Stance"')
    
    return new_sent

In [16]:
df['prompted_rep_3_w_modif_MC_corrected'] = df.apply(lambda x: replace_mc(x), axis=1)

In [17]:
df['prompted_rep_3_w_modif_MC_corrected'][0]

'How is the component best described?: "Stance", "Claim" or "Premise". Component: Topic: Should students be taught to compete or to cooperate?, Sentence: From this point of view, I firmly believe that we should attach more importance to cooperation during primary education., First or last in essay: Yes, First in paragraph: Yes, Last in paragraph: Yes, In in introduction: Yes, Is in conclusion: No. Part Of Speech tags: PRON, VERB, VERB, ADJ, NOUN, ADP, NOUN, ADP, ADJ, NOUN. Stance.'

In [18]:
# now do a sanity check to see if it's correct

In [19]:
last_tokens_list = []

In [21]:
def get_last_token(x):
    
    prompted_rep = x.prompted_rep_3_w_modif_MC_corrected
    
    last_token = prompted_rep.split()[-1]
    
    last_tokens_list.append(last_token)

In [22]:
df.apply(lambda x: get_last_token(x), axis=1)

0       None
1       None
2       None
3       None
4       None
        ... 
5968    None
5969    None
5970    None
5971    None
5972    None
Length: 5973, dtype: object

In [23]:
last_tokens_list

['Stance.',
 'Stance.',
 'Claim.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Claim.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Claim.',
 'Stance.',
 'Stance.',
 'Premise.',
 'Claim.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Claim.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Claim.',
 'Stance.',
 'Stance.',
 'Claim.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Claim.',
 'Premise.',
 'Premise.',
 'Stance.',
 'Stance.',
 'Claim.',
 'Claim.',
 'Claim.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Stance.',
 'Stance.',
 'Claim.',
 'Claim.',
 'Claim.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Claim.',
 'Stance.',
 'Stance.',
 'Claim.',
 'Claim.',
 'Claim.',
 'Claim.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Stance.',
 'Claim.',
 'Premise.',
 'Premise.',


In [24]:
set(last_tokens_list)

{'Claim.', 'Premise.', 'Stance.'}

In [25]:
####### YAAAAAAAAASSSSSSSSSS KINGGGGGGGGGGGGGGGG # correct.

## make train, test datasets

### train dataset

In [26]:
df_train = df[df.split == 'TRAIN']

In [27]:
df_train = df_train.reset_index(drop=True)

In [28]:
df_train

,essay_nr,component_id,label_and_comp_idxs,text,label_x,label_ComponentType,relation_SupportAttack,label_RelationType,label_LinkedNotLinked,split,...,structural_fts_as_text_combined,para_ratio,first_or_last,strct_fts_w_position_in_essay,component_pos_tags,strct_fts_essay_position_pos_tags,prompted_representation_2,prompted_representation_3,prompted_rep_3_w_modif_MC,prompted_rep_3_w_modif_MC_corrected
0,essay001,T1,MajorClaim 503 575,we should attach more importance to cooperatio...,MajorClaim,MajorClaim,[],,Linked,TRAIN,...,Topic: Should students be taught to compete or...,0.25,1,Topic: Should students be taught to compete or...,"Part Of Speech tags: PRON, VERB, VERB, ADJ, NO...",Topic: Should students be taught to compete or...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl...","How is the component best described?: ""MajorCl...","How is the component best described?: ""Stance""..."
1,essay001,T2,MajorClaim 2154 2231,a more cooperative attitudes towards life is m...,MajorClaim,MajorClaim,[],,Linked,TRAIN,...,Topic: Should students be taught to compete or...,1.00,1,Topic: Should students be taught to compete or...,"Part Of Speech tags: DET, ADV, ADJ, NOUN, ADP,...",Topic: Should students be taught to compete or...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl...","How is the component best described?: ""MajorCl...","How is the component best described?: ""Stance""..."
2,essay001,T3,Claim 591 714,"through cooperation, children can learn about ...",Claim,Claim,[],Support,Linked,TRAIN,...,Topic: Should students be taught to compete or...,0.50,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: ADP, NOUN, PUNCT, NOUN, V...",Topic: Should students be taught to compete or...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl...","How is the component best described?: ""MajorCl...","How is the component best described?: ""Stance""..."
3,essay001,T4,Premise 716 851,What we acquired from team work is not only ho...,Premise,Premise,[],Support,NotLinked,TRAIN,...,Topic: Should students be taught to compete or...,0.50,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: PRON, PRON, VERB, ADP, NO...",Topic: Should students be taught to compete or...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl...","How is the component best described?: ""MajorCl...","How is the component best described?: ""Stance""..."
4,essay001,T5,Premise 853 1086,"During the process of cooperation, children ca...",Premise,Premise,[],Support,NotLinked,TRAIN,...,Topic: Should students be taught to compete or...,0.50,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: ADP, DET, NOUN, ADP, NOUN...",Topic: Should students be taught to compete or...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl...","How is the component best described?: ""MajorCl...","How is the component best described?: ""Stance""..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4708,essay402,T11,Premise 1275 1339,indirectly they will learn how to socialize ea...,Premise,Premise,[],Support,NotLinked,TRAIN,...,Topic: Children should studying hard or playin...,0.75,0,Topic: Children should studying hard or playin...,"Part Of Speech tags: ADV, PRON, VERB, VERB, AD...",Topic: Children should studying hard or playin...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl...","How is the component best described?: ""MajorCl...","How is the component best described?: ""Stance""..."
4709,essay402,T12,Premise 1341 1388,That will make children getting lots of friends,Premise,Premise,[],Support,NotLinked,TRAIN,...,Topic: Children should studying hard or playin...,0.75,0,Topic: Children should studying hard or pl

In [29]:
df_train.split.value_counts()

TRAIN    4713
Name: split, dtype: int64

In [30]:
prompted_texts_train_full = df_train['prompted_rep_3_w_modif_MC_corrected'][:].tolist()

In [31]:
len(prompted_texts_train_full)

4713

In [32]:
4713 * .2

942.6

In [33]:
import random

In [34]:
prompted_texts_val = random.sample(prompted_texts_train_full, 472)

In [35]:
len(prompted_texts_val)

472

In [36]:
prompted_texts_train = [x for x in prompted_texts_train_full if x not in prompted_texts_val]

In [37]:
len(prompted_texts_train)

4235

In [38]:
len(prompted_texts_train) + len(prompted_texts_val)

4707

In [39]:
### prepare train dataset

In [40]:
inputs_train = tokenizer(prompted_texts_train, return_tensors='pt', max_length=512, truncation=True, padding='max_length')

In [41]:
inputs_train

{'input_ids': tensor([[ 101, 2129, 2003,  ...,    0,    0,    0],
        [ 101, 2129, 2003,  ...,    0,    0,    0],
        [ 101, 2129, 2003,  ...,    0,    0,    0],
        ...,
        [ 101, 2129, 2003,  ...,    0,    0,    0],
        [ 101, 2129, 2003,  ...,    0,    0,    0],
        [ 101, 2129, 2003,  ...,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [42]:
inputs_train['input_ids'].shape

torch.Size([4235, 512])

In [43]:
# for training we will need labels

In [44]:
inputs_train['labels'] = inputs_train.input_ids.detach().clone()

In [45]:
inputs_train

{'input_ids': tensor([[ 101, 2129, 2003,  ...,    0,    0,    0],
        [ 101, 2129, 2003,  ...,    0,    0,    0],
        [ 101, 2129, 2003,  ...,    0,    0,    0],
        ...,
        [ 101, 2129, 2003,  ...,    0,    0,    0],
        [ 101, 2129, 2003,  ...,    0,    0,    0],
        [ 101, 2129, 2003,  ...,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([[ 101, 2129, 2003,  ...,    0,    0,    0],
        [ 101, 2129, 2003,  ...,    0,    0,    0],
        [ 101, 2129, 2003,  ...,    0,    0,    0],
        ...,
      

In [46]:
inputs_train['labels'].shape

torch.Size([4235, 512])

In [47]:
# find where the last 102 is

In [48]:
(inputs_train['input_ids'][0] == 102).nonzero(as_tuple=True)[0].item()

124

In [49]:
tokenizer.decode(4366)

'c l a i m'

In [50]:
tokenizer.decode(18458)

'p r e m i s e'

In [51]:
# claim = 4366, premise = 18458

In [52]:
(inputs_train['input_ids'][1] == 102).nonzero(as_tuple=True)[0].item()

149

In [53]:
inputs_train['input_ids'][1][149]

tensor(102)

In [54]:
# confirm that all at [len-2] indices are either claim or premise

In [55]:
sep_indices_list = []

for i in range(len(prompted_texts_train)):
    sep_idx = (inputs_train['input_ids'][i] == 102).nonzero(as_tuple=True)[0].item()
    sep_indices_list.append(sep_idx)

In [56]:
sep_indices_list

[124,
 149,
 148,
 174,
 223,
 153,
 133,
 155,
 291,
 117,
 143,
 157,
 146,
 116,
 191,
 142,
 127,
 152,
 122,
 132,
 179,
 145,
 138,
 165,
 143,
 182,
 154,
 134,
 122,
 144,
 131,
 128,
 134,
 224,
 210,
 164,
 156,
 139,
 122,
 160,
 130,
 147,
 139,
 201,
 183,
 173,
 151,
 173,
 146,
 173,
 168,
 130,
 107,
 124,
 110,
 113,
 95,
 114,
 136,
 158,
 134,
 103,
 101,
 131,
 134,
 159,
 105,
 129,
 143,
 141,
 131,
 180,
 115,
 155,
 120,
 132,
 136,
 146,
 118,
 129,
 108,
 116,
 119,
 151,
 160,
 166,
 120,
 127,
 102,
 164,
 119,
 133,
 163,
 133,
 120,
 105,
 118,
 112,
 129,
 99,
 133,
 234,
 105,
 123,
 113,
 148,
 147,
 173,
 153,
 126,
 119,
 122,
 105,
 110,
 145,
 98,
 189,
 157,
 133,
 135,
 137,
 127,
 131,
 153,
 98,
 142,
 125,
 130,
 152,
 108,
 152,
 119,
 122,
 158,
 91,
 107,
 101,
 108,
 150,
 146,
 115,
 168,
 149,
 154,
 153,
 119,
 199,
 140,
 107,
 148,
 148,
 180,
 139,
 148,
 129,
 131,
 177,
 130,
 141,
 192,
 122,
 212,
 127,
 150,
 121,
 103,
 140,
 14

In [57]:
len(sep_indices_list)

4235

In [58]:
class_tokens_indices_list = [x - 2 for x in sep_indices_list]

In [59]:
class_tokens_indices_list

[122,
 147,
 146,
 172,
 221,
 151,
 131,
 153,
 289,
 115,
 141,
 155,
 144,
 114,
 189,
 140,
 125,
 150,
 120,
 130,
 177,
 143,
 136,
 163,
 141,
 180,
 152,
 132,
 120,
 142,
 129,
 126,
 132,
 222,
 208,
 162,
 154,
 137,
 120,
 158,
 128,
 145,
 137,
 199,
 181,
 171,
 149,
 171,
 144,
 171,
 166,
 128,
 105,
 122,
 108,
 111,
 93,
 112,
 134,
 156,
 132,
 101,
 99,
 129,
 132,
 157,
 103,
 127,
 141,
 139,
 129,
 178,
 113,
 153,
 118,
 130,
 134,
 144,
 116,
 127,
 106,
 114,
 117,
 149,
 158,
 164,
 118,
 125,
 100,
 162,
 117,
 131,
 161,
 131,
 118,
 103,
 116,
 110,
 127,
 97,
 131,
 232,
 103,
 121,
 111,
 146,
 145,
 171,
 151,
 124,
 117,
 120,
 103,
 108,
 143,
 96,
 187,
 155,
 131,
 133,
 135,
 125,
 129,
 151,
 96,
 140,
 123,
 128,
 150,
 106,
 150,
 117,
 120,
 156,
 89,
 105,
 99,
 106,
 148,
 144,
 113,
 166,
 147,
 152,
 151,
 117,
 197,
 138,
 105,
 146,
 146,
 178,
 137,
 146,
 127,
 129,
 175,
 128,
 139,
 190,
 120,
 210,
 125,
 148,
 119,
 101,
 138,
 145,

In [60]:
# inputs_pe['input_ids'][0][143].item() # correct

In [61]:
# now get a list of input ids at all these indics: they should be either claim = 4366, premise = 18458

In [62]:
list_index_minus_2 = []

for idx, val in enumerate(class_tokens_indices_list):
    at_idx_minus_2 = inputs_train['input_ids'][idx][val].item()
    list_index_minus_2.append(at_idx_minus_2)

In [63]:
set(list_index_minus_2)

{4366, 11032, 18458}

In [64]:
tokenizer.decode(11032)

's t a n c e'

In [65]:
# correct

In [66]:
# now make the class token indices 103

In [67]:
for idx, val in enumerate(class_tokens_indices_list):
    inputs_train['input_ids'][idx][val] = 103

In [68]:
# now check if they are 103

In [69]:
unsontroi_check_list = []

for idx, val in enumerate(class_tokens_indices_list):
    at_idx_minus_2 = inputs_train['input_ids'][idx][val].item()
    unsontroi_check_list.append(at_idx_minus_2)

In [70]:
set(unsontroi_check_list)

{103}

In [71]:
# correct

In [72]:
class PeDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [73]:
train_dataset = PeDataset(inputs_train)

In [74]:
# train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=)
# maybe we don't need this.

### prepare val dataset

In [75]:
inputs_eval = tokenizer(prompted_texts_val, return_tensors='pt', max_length=512, truncation=True, padding='max_length')

In [76]:
inputs_eval

{'input_ids': tensor([[ 101, 2129, 2003,  ...,    0,    0,    0],
        [ 101, 2129, 2003,  ...,    0,    0,    0],
        [ 101, 2129, 2003,  ...,    0,    0,    0],
        ...,
        [ 101, 2129, 2003,  ...,    0,    0,    0],
        [ 101, 2129, 2003,  ...,    0,    0,    0],
        [ 101, 2129, 2003,  ...,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [77]:
inputs_eval['input_ids'].shape

torch.Size([472, 512])

In [78]:
# for training we will need labels

In [79]:
inputs_eval['labels'] = inputs_eval.input_ids.detach().clone()

In [80]:
inputs_eval

{'input_ids': tensor([[ 101, 2129, 2003,  ...,    0,    0,    0],
        [ 101, 2129, 2003,  ...,    0,    0,    0],
        [ 101, 2129, 2003,  ...,    0,    0,    0],
        ...,
        [ 101, 2129, 2003,  ...,    0,    0,    0],
        [ 101, 2129, 2003,  ...,    0,    0,    0],
        [ 101, 2129, 2003,  ...,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([[ 101, 2129, 2003,  ...,    0,    0,    0],
        [ 101, 2129, 2003,  ...,    0,    0,    0],
        [ 101, 2129, 2003,  ...,    0,    0,    0],
        ...,
      

In [81]:
inputs_eval['labels'].shape

torch.Size([472, 512])

In [82]:
# find where the last 102 is

In [83]:
(inputs_eval['input_ids'][0] == 102).nonzero(as_tuple=True)[0].item()

151

In [84]:
tokenizer.decode(4366)

'c l a i m'

In [85]:
tokenizer.decode(18458)

'p r e m i s e'

In [86]:
# claim = 4366, premise = 18458

In [87]:
(inputs_eval['input_ids'][1] == 102).nonzero(as_tuple=True)[0].item()

130

In [88]:
inputs_eval['input_ids'][1][130]

tensor(102)

In [89]:
# confirm that all at [len-2] indices are either claim or premise

In [90]:
sep_indices_list = []

for i in range(len(prompted_texts_val)):
    sep_idx = (inputs_eval['input_ids'][i] == 102).nonzero(as_tuple=True)[0].item()
    sep_indices_list.append(sep_idx)

In [91]:
sep_indices_list

[151,
 130,
 190,
 220,
 158,
 107,
 121,
 127,
 145,
 162,
 142,
 144,
 150,
 113,
 141,
 128,
 109,
 110,
 111,
 106,
 122,
 150,
 137,
 117,
 140,
 104,
 122,
 136,
 136,
 140,
 144,
 111,
 162,
 162,
 152,
 208,
 120,
 204,
 133,
 109,
 134,
 104,
 145,
 144,
 151,
 104,
 108,
 171,
 193,
 133,
 111,
 129,
 133,
 180,
 170,
 137,
 112,
 103,
 161,
 182,
 143,
 192,
 186,
 133,
 165,
 122,
 105,
 145,
 134,
 130,
 153,
 127,
 179,
 109,
 138,
 150,
 164,
 160,
 155,
 173,
 109,
 123,
 145,
 130,
 128,
 137,
 136,
 122,
 127,
 136,
 120,
 116,
 138,
 149,
 134,
 110,
 135,
 130,
 105,
 132,
 136,
 99,
 144,
 121,
 118,
 127,
 221,
 127,
 149,
 109,
 144,
 136,
 174,
 107,
 134,
 143,
 133,
 114,
 120,
 150,
 184,
 202,
 128,
 109,
 143,
 129,
 128,
 129,
 126,
 165,
 112,
 112,
 141,
 121,
 118,
 110,
 117,
 142,
 123,
 109,
 162,
 152,
 126,
 102,
 163,
 142,
 120,
 148,
 146,
 175,
 186,
 177,
 179,
 110,
 131,
 120,
 159,
 136,
 116,
 145,
 134,
 104,
 174,
 179,
 157,
 146,
 115,

In [92]:
len(sep_indices_list)

472

In [93]:
class_tokens_indices_list = [x - 2 for x in sep_indices_list]

In [94]:
class_tokens_indices_list

[149,
 128,
 188,
 218,
 156,
 105,
 119,
 125,
 143,
 160,
 140,
 142,
 148,
 111,
 139,
 126,
 107,
 108,
 109,
 104,
 120,
 148,
 135,
 115,
 138,
 102,
 120,
 134,
 134,
 138,
 142,
 109,
 160,
 160,
 150,
 206,
 118,
 202,
 131,
 107,
 132,
 102,
 143,
 142,
 149,
 102,
 106,
 169,
 191,
 131,
 109,
 127,
 131,
 178,
 168,
 135,
 110,
 101,
 159,
 180,
 141,
 190,
 184,
 131,
 163,
 120,
 103,
 143,
 132,
 128,
 151,
 125,
 177,
 107,
 136,
 148,
 162,
 158,
 153,
 171,
 107,
 121,
 143,
 128,
 126,
 135,
 134,
 120,
 125,
 134,
 118,
 114,
 136,
 147,
 132,
 108,
 133,
 128,
 103,
 130,
 134,
 97,
 142,
 119,
 116,
 125,
 219,
 125,
 147,
 107,
 142,
 134,
 172,
 105,
 132,
 141,
 131,
 112,
 118,
 148,
 182,
 200,
 126,
 107,
 141,
 127,
 126,
 127,
 124,
 163,
 110,
 110,
 139,
 119,
 116,
 108,
 115,
 140,
 121,
 107,
 160,
 150,
 124,
 100,
 161,
 140,
 118,
 146,
 144,
 173,
 184,
 175,
 177,
 108,
 129,
 118,
 157,
 134,
 114,
 143,
 132,
 102,
 172,
 177,
 155,
 144,
 113,

In [95]:
# inputs_pe['input_ids'][0][143].item() # correct

In [96]:
# now get a list of input ids at all these indics: they should be either claim = 4366, premise = 18458

In [97]:
list_index_minus_2 = []

for idx, val in enumerate(class_tokens_indices_list):
    at_idx_minus_2 = inputs_eval['input_ids'][idx][val].item()
    list_index_minus_2.append(at_idx_minus_2)

In [98]:
set(list_index_minus_2)

{4366, 11032, 18458}

In [99]:
tokenizer.decode(11032)

's t a n c e'

In [100]:
# correct

In [101]:
# now make the class token indices 103

In [102]:
for idx, val in enumerate(class_tokens_indices_list):
    inputs_eval['input_ids'][idx][val] = 103

In [103]:
# now check if they are 103

In [104]:
unsontroi_check_list = []

for idx, val in enumerate(class_tokens_indices_list):
    at_idx_minus_2 = inputs_eval['input_ids'][idx][val].item()
    unsontroi_check_list.append(at_idx_minus_2)

In [105]:
set(unsontroi_check_list)

{103}

In [106]:
# correct

In [107]:
# class PeDataset(torch.utils.data.Dataset):
#     def __init__(self, encodings):
#         self.encodings = encodings
#     def __getitem__(self, idx):
#         return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
#     def __len__(self):
#         return len(self.encodings.input_ids)

In [108]:
eval_dataset = PeDataset(inputs_eval)

### test dataset

In [109]:
df_test = df[df.split == 'TEST']

In [110]:
df_test = df_test.reset_index(drop=True)

In [111]:
df_test

,essay_nr,component_id,label_and_comp_idxs,text,label_x,label_ComponentType,relation_SupportAttack,label_RelationType,label_LinkedNotLinked,split,...,structural_fts_as_text_combined,para_ratio,first_or_last,strct_fts_w_position_in_essay,component_pos_tags,strct_fts_essay_position_pos_tags,prompted_representation_2,prompted_representation_3,prompted_rep_3_w_modif_MC,prompted_rep_3_w_modif_MC_corrected
0,essay004,T1,MajorClaim 262 376,this industry has affected the cultural attrib...,MajorClaim,MajorClaim,[],,Linked,TEST,...,Topic: International tourism is now more commo...,0.25,1,Topic: International tourism is now more commo...,"Part Of Speech tags: DET, NOUN, AUX, VERB, DET...",Topic: International tourism is now more commo...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl...","How is the component best described?: ""MajorCl...","How is the component best described?: ""Stance""..."
1,essay004,T2,MajorClaim 1663 1758,the tourism has created threatening pressure o...,MajorClaim,MajorClaim,[],,Linked,TEST,...,Topic: International tourism is now more commo...,1.00,1,Topic: International tourism is now more commo...,"Part Of Speech tags: DET, NOUN, AUX, VERB, VER...",Topic: International tourism is now more commo...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl...","How is the component best described?: ""MajorCl...","How is the component best described?: ""Stance""..."
2,essay004,T3,Claim 179 239,the tourism bring large profit for the destina...,Claim,Claim,[],Attack,NotLinked,TEST,...,Topic: International tourism is now more commo...,0.25,1,Topic: International tourism is now more commo...,"Part Of Speech tags: DET, NOUN, VERB, ADJ, NOU...",Topic: International tourism is now more commo...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl...","How is the component best described?: ""MajorCl...","How is the component best described?: ""Stance""..."
3,essay004,T4,Claim 953 1031,international tourism can create negative impa...,Claim,Claim,[],Support,Linked,TEST,...,Topic: International tourism is now more commo...,0.50,0,Topic: International tourism is now more commo...,"Part Of Speech tags: ADJ, NOUN, VERB, VERB, AD...",Topic: International tourism is now more commo...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl...","How is the component best described?: ""MajorCl...","How is the component best described?: ""Stance""..."
4,essay004,T5,Claim 1578 1624,tourism has threatened the nature environments,Claim,Claim,[],Support,Linked,TEST,...,Topic: International tourism is now more commo...,0.75,0,Topic: International tourism is now more commo...,"Part Of Speech tags: NOUN, AUX, VERB, DET, NOU...",Topic: International tourism is now more commo...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl...","How is the component best described?: ""MajorCl...","How is the component best described?: ""Stance""..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1255,essay398,T12,Claim 1484 1589,universities should encourage more girls to ch...,Claim,Claim,[],Support,Linked,TEST,...,Topic: We can not forcedly put the same number...,0.80,0,Topic: We can not forcedly put the same number...,"Part Of Speech tags: NOUN, VERB, VERB, ADJ, NO...",Topic: We can not forcedly put the same number...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl...","How is the component best described?: ""MajorCl...","How is the component best described?: ""Stance""..."
1256,essay398,T13,Premise 1595 1648,this could avoid imbalance of gender in some s...,Premise,Premise,[],Support,NotLinked,TEST,...,Topic: We can not forcedly put the same number...,0.80,0,Topic: We can not forcedly put the same number...,"Part Of Speech tags: DET, 

In [112]:
df_test.split.value_counts()

TEST    1260
Name: split, dtype: int64

In [114]:
prompted_texts_test = df_test['prompted_rep_3_w_modif_MC_corrected'][:].tolist()

In [115]:
len(prompted_texts_test)

1260

In [116]:
inputs_test = tokenizer(prompted_texts_test, return_tensors='pt', max_length=512, truncation=True, padding='max_length')

In [117]:
inputs_test

{'input_ids': tensor([[ 101, 2129, 2003,  ...,    0,    0,    0],
        [ 101, 2129, 2003,  ...,    0,    0,    0],
        [ 101, 2129, 2003,  ...,    0,    0,    0],
        ...,
        [ 101, 2129, 2003,  ...,    0,    0,    0],
        [ 101, 2129, 2003,  ...,    0,    0,    0],
        [ 101, 2129, 2003,  ...,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [118]:
inputs_test['input_ids'].shape

torch.Size([1260, 512])

In [117]:
# for test we do not need labels

In [118]:
#inputs_train['labels'] = inputs_train.input_ids.detach().clone()

In [119]:
#inputs_train

In [120]:
#inputs_train['labels'].shape

In [121]:
# find where the last 102 is

In [122]:
#(inputs_train['input_ids'][0] == 102).nonzero(as_tuple=True)[0].item()

In [123]:
#tokenizer.decode(4366)

In [124]:
#tokenizer.decode(18458)

In [125]:
# claim = 4366, premise = 18458

In [126]:
#(inputs_train['input_ids'][1] == 102).nonzero(as_tuple=True)[0].item()

In [127]:
#inputs_train['input_ids'][1][169]

In [128]:
# confirm that all at [len-2] indices are either claim or premise

In [119]:
sep_indices_list_test = []

for i in range(len(prompted_texts_test)):
    sep_idx = (inputs_test['input_ids'][i] == 102).nonzero(as_tuple=True)[0].item()
    sep_indices_list_test.append(sep_idx)

In [120]:
sep_indices_list_test

[149,
 131,
 132,
 115,
 102,
 141,
 257,
 148,
 126,
 234,
 153,
 118,
 144,
 157,
 140,
 143,
 203,
 150,
 134,
 109,
 116,
 117,
 120,
 97,
 144,
 131,
 137,
 141,
 108,
 137,
 109,
 122,
 125,
 111,
 147,
 138,
 106,
 141,
 104,
 130,
 130,
 114,
 117,
 125,
 118,
 97,
 105,
 121,
 116,
 91,
 101,
 164,
 126,
 147,
 107,
 139,
 154,
 144,
 106,
 118,
 145,
 148,
 151,
 130,
 161,
 141,
 194,
 177,
 223,
 168,
 168,
 142,
 122,
 142,
 115,
 131,
 146,
 113,
 135,
 141,
 125,
 150,
 158,
 148,
 134,
 106,
 112,
 144,
 158,
 105,
 102,
 109,
 122,
 132,
 122,
 159,
 135,
 149,
 119,
 127,
 109,
 185,
 98,
 174,
 145,
 141,
 143,
 131,
 119,
 229,
 138,
 150,
 146,
 111,
 133,
 185,
 150,
 135,
 114,
 126,
 133,
 163,
 191,
 132,
 184,
 100,
 202,
 100,
 155,
 114,
 160,
 128,
 138,
 166,
 116,
 159,
 113,
 132,
 97,
 130,
 166,
 115,
 134,
 110,
 140,
 163,
 199,
 117,
 116,
 122,
 145,
 183,
 132,
 131,
 128,
 135,
 154,
 110,
 150,
 161,
 210,
 141,
 118,
 128,
 148,
 142,
 118,
 14

In [121]:
len(sep_indices_list_test)

1260

In [122]:
class_tokens_indices_test_list = [x - 2 for x in sep_indices_list_test]

In [123]:
class_tokens_indices_test_list

[147,
 129,
 130,
 113,
 100,
 139,
 255,
 146,
 124,
 232,
 151,
 116,
 142,
 155,
 138,
 141,
 201,
 148,
 132,
 107,
 114,
 115,
 118,
 95,
 142,
 129,
 135,
 139,
 106,
 135,
 107,
 120,
 123,
 109,
 145,
 136,
 104,
 139,
 102,
 128,
 128,
 112,
 115,
 123,
 116,
 95,
 103,
 119,
 114,
 89,
 99,
 162,
 124,
 145,
 105,
 137,
 152,
 142,
 104,
 116,
 143,
 146,
 149,
 128,
 159,
 139,
 192,
 175,
 221,
 166,
 166,
 140,
 120,
 140,
 113,
 129,
 144,
 111,
 133,
 139,
 123,
 148,
 156,
 146,
 132,
 104,
 110,
 142,
 156,
 103,
 100,
 107,
 120,
 130,
 120,
 157,
 133,
 147,
 117,
 125,
 107,
 183,
 96,
 172,
 143,
 139,
 141,
 129,
 117,
 227,
 136,
 148,
 144,
 109,
 131,
 183,
 148,
 133,
 112,
 124,
 131,
 161,
 189,
 130,
 182,
 98,
 200,
 98,
 153,
 112,
 158,
 126,
 136,
 164,
 114,
 157,
 111,
 130,
 95,
 128,
 164,
 113,
 132,
 108,
 138,
 161,
 197,
 115,
 114,
 120,
 143,
 181,
 130,
 129,
 126,
 133,
 152,
 108,
 148,
 159,
 208,
 139,
 116,
 126,
 146,
 140,
 116,
 146,


In [124]:
# inputs_pe['input_ids'][0][143].item() # correct

In [125]:
# now get a list of input ids at all these indices: they should be either claim = 4366, premise = 18458

In [126]:
list_index_minus_2_test = []

for idx, val in enumerate(class_tokens_indices_test_list):
    at_idx_minus_2 = inputs_test['input_ids'][idx][val].item()
    list_index_minus_2_test.append(at_idx_minus_2)

In [127]:
set(list_index_minus_2)

{4366, 11032, 18458}

In [128]:
# correct

In [129]:
# now make the class token indices 103

In [130]:
for idx, val in enumerate(class_tokens_indices_test_list):
    inputs_test['input_ids'][idx][val] = 103

In [131]:
# now check if they are 103

In [132]:
unsontroi_check_list = []

for idx, val in enumerate(class_tokens_indices_test_list):
    at_idx_minus_2 = inputs_test['input_ids'][idx][val].item()
    unsontroi_check_list.append(at_idx_minus_2)

In [133]:
set(unsontroi_check_list)

{103}

In [134]:
# correct

In [135]:
# class PeDataset(torch.utils.data.Dataset):
#     def __init__(self, encodings):
#         self.encodings = encodings
#     def __getitem__(self, idx):
#         return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
#     def __len__(self):
#         return len(self.encodings.input_ids)

In [136]:
test_dataset = PeDataset(inputs_test)

In [137]:
# test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=True)
# maybe we don't need this.

## get the model

In [138]:
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [139]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [140]:
model.to(device)

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [141]:
# let's see with a simple trainer now and then we will try the fancy trainer.

### training

In [142]:
from transformers import TrainingArguments

In [143]:
import gc

In [144]:
gc.collect()
torch.cuda.empty_cache()

In [145]:
NB_EPOCHS = 10
BATCH_SIZE = 8

In [146]:
training_args = TrainingArguments(
    
    # output
    output_dir='output',          
    
    # params
    num_train_epochs=NB_EPOCHS,               # nb of epochs
    per_device_train_batch_size=BATCH_SIZE,   # batch size per device during training
    per_device_eval_batch_size=BATCH_SIZE,    # cf. paper Sun et al.
    learning_rate=1e-5,#2e-5,                 # cf. paper Sun et al.
#     warmup_steps=500,                         # number of warmup steps for learning rate scheduler
    warmup_ratio=0.1,                         # cf. paper Sun et al.
    weight_decay=0.01,                        # strength of weight decay
    
    # eval
    evaluation_strategy="epoch",              # cf. paper Sun et al.
    #eval_steps=20,                            # cf. paper Sun et al.
    #eval_accumulation_steps = BATCH_SIZE,
    # fp16_full_eval = True,
    
    # clean the tensor board folder to see the loss properly.
    
    # log
    logging_dir="/notebooks/Prompting/results",  
    logging_strategy='steps',
    logging_steps=100,
    
    # save
    save_strategy='epoch',
    save_total_limit=2,
    # save_steps=20, # default 500
    load_best_model_at_end=True,              # cf. paper Sun et al.
    metric_for_best_model='eval_loss' 
    # metric_for_best_model='f1'
)

In [147]:
# from transformers import TrainingArguments

# args = TrainingArguments(
#     output_dir = 'out',
#     per_device_train_batch_size=16,
#     num_train_epochs=2
# )

In [148]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
    #compute_metrics=compute_metrics
)

In [149]:
trainer.train()

***** Running training *****
  Num examples = 4235
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 5300
/tmp/ipykernel_700/1625588428.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch,Training Loss,Validation Loss
1,0.010000,0.003696
2,0.002000,0.001610
3,0.001600,0.001708
4,0.001200,0.001366
5,0.001000,0.001427
6,0.001000,0.001276
7,0.001000,0.001196
8,0.000900,0.001254
9,0.000900,0.001197
10,0.000900,0.001165


***** Running Evaluation *****
  Num examples = 472
  Batch size = 8
Saving model checkpoint to output/checkpoint-530
Configuration saved in output/checkpoint-530/config.json
Model weights saved in output/checkpoint-530/pytorch_model.bin
/tmp/ipykernel_700/1625588428.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 472
  Batch size = 8
Saving model checkpoint to output/checkpoint-1060
Configuration saved in output/checkpoint-1060/config.json
Model weights saved in output/checkpoint-1060/pytorch_model.bin
/tmp/ipykernel_700/1625588428.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.t

TrainOutput(global_step=5300, training_loss=0.2590845661835288, metrics={'train_runtime': 2935.5279, 'train_samples_per_second': 14.427, 'train_steps_per_second': 1.805, 'total_flos': 1.114672382208e+16, 'train_loss': 0.2590845661835288, 'epoch': 10.0})

In [150]:
trainer.save_model("pe_mask_model_prompted_rep_3")

Saving model checkpoint to pe_mask_model_prompted_rep_3
Configuration saved in pe_mask_model_prompted_rep_3/config.json
Model weights saved in pe_mask_model_prompted_rep_3/pytorch_model.bin


In [156]:
# new idea: take this model and finetune the PE dataset on it.
# so instead of loading a pre-trained BERT, load this instead.

### Pipeline

In [151]:
saved_model = BertForMaskedLM.from_pretrained("/notebooks/Prompting/notebooks/prompt_template_3/pe_mask_model_prompted_rep_3")

loading configuration file /notebooks/Prompting/notebooks/prompt_template_3/pe_mask_model_prompted_rep_3/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.12.5",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file /notebooks/Prompting/notebooks/prompt_template_3/pe_mask_model_prompted_rep_3/pytorch_model.bin
All model checkpoint weights were used when initializing BertForMaskedLM.

Al

In [152]:
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [153]:
from transformers import pipeline

In [154]:
classifier = pipeline("fill-mask", model=saved_model, tokenizer=tokenizer)

In [155]:
classifier.model

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [156]:
targets = ['stance', 'claim', 'premise']

In [157]:
# ok now see how to do this for the whole list of test components

In [158]:
# but first we must replace the last token by [MASK]!

In [159]:
def get_masked_components(x):
    
    prompted_rep = x.prompted_rep_3_w_modif_MC_corrected
    
    last_token = prompted_rep.split()[-1]
    

    new_rep = prompted_rep.replace(last_token, "[MASK].")

    
    return new_rep

In [160]:
df_test

,essay_nr,component_id,label_and_comp_idxs,text,label_x,label_ComponentType,relation_SupportAttack,label_RelationType,label_LinkedNotLinked,split,...,structural_fts_as_text_combined,para_ratio,first_or_last,strct_fts_w_position_in_essay,component_pos_tags,strct_fts_essay_position_pos_tags,prompted_representation_2,prompted_representation_3,prompted_rep_3_w_modif_MC,prompted_rep_3_w_modif_MC_corrected
0,essay004,T1,MajorClaim 262 376,this industry has affected the cultural attrib...,MajorClaim,MajorClaim,[],,Linked,TEST,...,Topic: International tourism is now more commo...,0.25,1,Topic: International tourism is now more commo...,"Part Of Speech tags: DET, NOUN, AUX, VERB, DET...",Topic: International tourism is now more commo...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl...","How is the component best described?: ""MajorCl...","How is the component best described?: ""Stance""..."
1,essay004,T2,MajorClaim 1663 1758,the tourism has created threatening pressure o...,MajorClaim,MajorClaim,[],,Linked,TEST,...,Topic: International tourism is now more commo...,1.00,1,Topic: International tourism is now more commo...,"Part Of Speech tags: DET, NOUN, AUX, VERB, VER...",Topic: International tourism is now more commo...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl...","How is the component best described?: ""MajorCl...","How is the component best described?: ""Stance""..."
2,essay004,T3,Claim 179 239,the tourism bring large profit for the destina...,Claim,Claim,[],Attack,NotLinked,TEST,...,Topic: International tourism is now more commo...,0.25,1,Topic: International tourism is now more commo...,"Part Of Speech tags: DET, NOUN, VERB, ADJ, NOU...",Topic: International tourism is now more commo...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl...","How is the component best described?: ""MajorCl...","How is the component best described?: ""Stance""..."
3,essay004,T4,Claim 953 1031,international tourism can create negative impa...,Claim,Claim,[],Support,Linked,TEST,...,Topic: International tourism is now more commo...,0.50,0,Topic: International tourism is now more commo...,"Part Of Speech tags: ADJ, NOUN, VERB, VERB, AD...",Topic: International tourism is now more commo...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl...","How is the component best described?: ""MajorCl...","How is the component best described?: ""Stance""..."
4,essay004,T5,Claim 1578 1624,tourism has threatened the nature environments,Claim,Claim,[],Support,Linked,TEST,...,Topic: International tourism is now more commo...,0.75,0,Topic: International tourism is now more commo...,"Part Of Speech tags: NOUN, AUX, VERB, DET, NOU...",Topic: International tourism is now more commo...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl...","How is the component best described?: ""MajorCl...","How is the component best described?: ""Stance""..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1255,essay398,T12,Claim 1484 1589,universities should encourage more girls to ch...,Claim,Claim,[],Support,Linked,TEST,...,Topic: We can not forcedly put the same number...,0.80,0,Topic: We can not forcedly put the same number...,"Part Of Speech tags: NOUN, VERB, VERB, ADJ, NO...",Topic: We can not forcedly put the same number...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl...","How is the component best described?: ""MajorCl...","How is the component best described?: ""Stance""..."
1256,essay398,T13,Premise 1595 1648,this could avoid imbalance of gender in some s...,Premise,Premise,[],Support,NotLinked,TEST,...,Topic: We can not forcedly put the same number...,0.80,0,Topic: We can not forcedly put the same number...,"Part Of Speech tags: DET, 

In [161]:
df_test['masked_representation_3_for_pipeline'] = df_test.apply(lambda x: get_masked_components(x), axis=1)

In [162]:
df_test['masked_representation_3_for_pipeline'][0]

'How is the component best described?: "Stance", "Claim" or "Premise". Component: Topic: International tourism is now more common than ever before, Sentence: While some might think the tourism bring large profit for the destination countries, I would contend that this industry has affected the cultural attributes and damaged the natural environment of the tourist destinations., First or last in essay: Yes, First in paragraph: No, Last in paragraph: Yes, In in introduction: Yes, Is in conclusion: No. Part Of Speech tags: DET, NOUN, AUX, VERB, DET, ADJ, NOUN, CCONJ, VERB, DET, ADJ, NOUN, ADP, DET, NOUN, NOUN. [MASK].'

In [163]:
# ok now do a sanity check to see that they are done correctly

In [164]:
last_tokens_mask_list = []

In [165]:
def get_last_token(x):
    
    prompted_rep = x.masked_representation_3_for_pipeline
    
    last_token = prompted_rep.split()[-1]
    
    last_tokens_mask_list.append(last_token)

In [166]:
df_test.apply(lambda x: get_last_token(x), axis=1)

0       None
1       None
2       None
3       None
4       None
        ... 
1255    None
1256    None
1257    None
1258    None
1259    None
Length: 1260, dtype: object

In [167]:
last_tokens_mask_list

['[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[M

In [168]:
set(last_tokens_mask_list)

{'[MASK].'}

In [169]:
###### YASSSSS SIRRRRRRRRRRR # correct

In [170]:
# ok now see how we can give all these 1260 statements to the pipeline

In [171]:
test_components_list = df_test['masked_representation_3_for_pipeline'][:].tolist()

In [172]:
len(test_components_list)

1260

In [173]:
# ok now give it to the pipeline

In [174]:
classifier_output = classifier(test_components_list, targets=targets)

Disabling tokenizer parallelism, we're using DataLoader multithreading already


In [175]:
classifier_output[0][0]['token']

11032

In [176]:
predicted_labels_test = []

for idx in range(1260):
    
    predicted_token = classifier_output[idx][0]['token']
    predicted_labels_test.append(predicted_token)

In [177]:
len(predicted_labels_test)

1260

In [178]:
# sanity check
set(predicted_labels_test)

{4366, 11032, 18458}

In [179]:
# ok correct

In [180]:
test_labels_str = df_test.label_ComponentType[:].tolist()

In [181]:
test_labels_str

['MajorClaim',
 'MajorClaim',
 'Claim',
 'Claim',
 'Claim',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'MajorClaim',
 'MajorClaim',
 'Claim',
 'Claim',
 'Claim',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Claim',
 'MajorClaim',
 'MajorClaim',
 'Claim',
 'Claim',
 'Claim',
 'Claim',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'MajorClaim',
 'MajorClaim',
 'Claim',
 'Premise',
 'Premise',
 'Claim',
 'Claim',
 'Claim',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'MajorClaim',
 'MajorClaim',
 'Claim',
 'Claim',
 'Claim',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'MajorClaim',
 'Claim',
 'Claim',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',

In [182]:
set(test_labels_str)

{'Claim', 'MajorClaim', 'Premise'}

In [183]:
test_labels_int = []

In [184]:
def get_int_labels():
    
    for idx, val in enumerate(test_labels_str):
        
        if val == 'MajorClaim':
            
            test_labels_int.append(11032)
            
        elif val == 'Claim':
            
            test_labels_int.append(4366)
            
        elif val == 'Premise':
            
            test_labels_int.append(18458)

In [185]:
get_int_labels()

In [186]:
test_labels_int

[11032,
 11032,
 4366,
 4366,
 4366,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 11032,
 11032,
 4366,
 4366,
 4366,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 4366,
 11032,
 11032,
 4366,
 4366,
 4366,
 4366,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 11032,
 11032,
 4366,
 18458,
 18458,
 4366,
 4366,
 4366,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 11032,
 11032,
 4366,
 4366,
 4366,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 11032,
 4366,
 4366,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 4366,
 11032,
 4366,
 11032,
 4366,
 4366,
 4366,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 11032,
 11032,
 4366,
 4366,
 4366,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 11032,
 11032,
 4366,
 4366,
 4366,
 18458,
 18458,
 18458,
 18458,
 11032,
 4366,
 11032,


In [187]:
len(test_labels_int)

1260

In [188]:
set(test_labels_int)

{4366, 11032, 18458}

In [189]:
# now do the classification report

In [190]:
from sklearn.metrics import classification_report

In [191]:
print(classification_report(test_labels_int, predicted_labels_test, digits=3))

              precision    recall  f1-score   support

        4366      0.678     0.689     0.683       302
       11032      0.751     0.967     0.846       153
       18458      0.935     0.878     0.906       805

    accuracy                          0.844      1260
   macro avg      0.788     0.845     0.812      1260
weighted avg      0.851     0.844     0.845      1260

